In [ ]:
# %pip install bm25s
# %pip install spacy
# %pip install -U 'spacy[cuda12x]'
# %pip install rouge_score
# %pip install pysbd

In [2]:
import functions as fct
import time
import os
import pandas as pd

# BM25

In [ ]:
# # Définition des chemins vers les fichiers JSON d'entraînement et de développement
# train_path = 'SCOTUS/train.json'
# dev_path = 'SCOTUS/dev.json'

# # Ouverture du fichier d'entraînement
# train = fct.open_file(train_path, "json")

# # Récupération du document source du premier élément de l'ensemble d'entraînement
# # document = train[0]["raw_source"]
# document = train

# # Récupération des éléments de la cible (faits, question, conclusion)
# paragraph_target = (
#     train[0]['raw_target']['facts_of_the_case'] +
#     train[0]['raw_target']['question'] +
#     train[0]['raw_target']['conclusion']
# )

# train_path = 'data_txt_save/train_0.txt'
# train = fct.open_file(train_path, "txt")

# document = train


# # Segmentation des phrases du document source
# sentences = fct.sent_segmentation(document, method='custom_spacy')

# # Résumé des phrases en utilisant le modèle BERT
# summary = fct.bb25LegalSum(sentences, "bert-base-uncased", 5)

# # Évaluation de la qualité du résumé à l'aide de la métrique ROUGE
# bb25_rouge = fct.rouge_evaluations(" ".join(summary), paragraph_target)

In [ ]:
# bb25_rouge

In [1]:
# Définition des chemins vers les fichiers JSON d'entraînement et de développement
train_path_json = 'SCOTUS/train.json'
dev_path_json = 'SCOTUS/dev.json'

# Ouverture du fichier d'entraînement JSON
train_json = fct.open_file(train_path_json, "json")

text_number = 0

# Récupération du document source et des éléments de la cible (faits, question, conclusion)
document_json = train_json[text_number]["raw_source"]
paragraph_target_json = (
    train_json[text_number]['raw_target']['facts_of_the_case'] +
    train_json[text_number]['raw_target']['question'] +
    train_json[text_number]['raw_target']['conclusion']
)

# Définir le chemin du fichier texte à traiter
clean_path_txt = f'clean_data_txt_save/train_{text_number}.txt'

# Ouverture du fichier d'entraînement TXT
clean_document_txt = fct.open_file(clean_path_txt, "txt")

# Liste des méthodes de segmentation et des modèles à tester
methods = ['nltk', 'spacy', 'pySBD', 'custom_spacy']  # Méthodes de segmentation à tester
methods = ['pySBD']  

model = "bert-base-uncased"  # Modèle de résumé à utiliser

def save_txt(text, folder_name, file_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    file_path = os.path.join(folder_name, file_name)

    with open(file_path, 'w') as file:
        file.write(text)

# Fonction pour évaluer les modèles
def evaluate_models(document, paragraph_target, file_type):
    results = pd.DataFrame()
    
    for method in methods:
        start_time = time.time()

        # Segmentation des phrases du document source
        sentences = fct.sent_segmentation(document, method=method)
        
        # Résumé des phrases
        query = fct.select_query(document)
        summary = fct.bb25LegalSum(sentences, model, query)
        
        # Évaluation de la qualité du résumé à l'aide de la métrique ROUGE
        bb25_rouge = fct.rouge_evaluations(" ".join(summary), paragraph_target)

        end_time = time.time()
        execution_time = end_time - start_time
                
        result = bb25_rouge.set_index(' ').T
        result.insert(0, 'Method', file_type+'_'+method)
        result['Execution time'] = execution_time
        
        summary_conc = "\n".join(summary)
        save_txt(summary_conc, f"summaries-2/{method}/", f"summary_train_{text_number}.txt") #save train_n summary
        
        results = pd.concat([results, result], ignore_index=True)
        
    return results, summary

results = pd.DataFrame()

# Évaluation des modèles pour le fichier JSON
# r, _ = evaluate_models(document_json, paragraph_target_json, "JSON")
# results = pd.concat([results, r], ignore_index=True)

result, summary = evaluate_models(clean_document_txt, paragraph_target_json, "Cleaned TXT")  # Utilise le même paragraphe cible
results = pd.concat([results, result], ignore_index=True)

NameError: name 'fct' is not defined

In [1]:
import functions as fct
import time
import os
import pandas as pd
from tqdm import tqdm

train_path_json = 'SCOTUS/train.json'

train_json = fct.open_file(train_path_json, "json")

def save_txt(text, folder_name, file_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    file_path = os.path.join(folder_name, file_name)

    with open(file_path, 'w') as file:
        file.write(text)

# Fonction pour évaluer les modèles
def evaluate_models(document, paragraph_target, file_type):  
    start_time = time.time()
    sentences = fct.sent_segmentation(document, method=method)
    query = fct.select_query(document)
    return query


method = 'pySBD'
model = "bert-base-uncased" 

results = []

range_start = 0
range_end = 200

for text_number in tqdm(range(range_start, range_end)):
    document_json = train_json[text_number]["raw_source"]
    paragraph_target_json = (
        train_json[text_number]['raw_target']['facts_of_the_case'] +
        train_json[text_number]['raw_target']['question'] +
        train_json[text_number]['raw_target']['conclusion']
    )

    clean_path_txt = f'clean_data_txt_save/train_{text_number}.txt'
    clean_document_txt = fct.open_file(clean_path_txt, "txt")

    query = evaluate_models(clean_document_txt, paragraph_target_json, "Cleaned TXT") 
    
    results.append((text_number, query))

100%|██████████| 200/200 [01:03<00:00,  3.16it/s]


In [4]:
for (i, query) in results:
    # if query == 'law and legal rights':
    #     print(i)
        
    print(f'{i} : {query}')

0 : A California jury convicted respondent of,  inter alia, murder and attempted murder. 
1 : Seth P. Waxman argued the cause for petitioners. 
2 : In Adarand Constructors, Inc. v. Pena,  515 U. S. 200  (AdarandI), this Court held that strict scrutiny governs whether race-basedclassifications violate equal protection and remanded for adetermination whether the race-based components of the Departmentof Transportation's (DOT's) Disadvantaged Business Enterprise (DBE)program could survive such review. 
3 : An Iowa policeman stopped petitioner Knowles for speeding and issued him a citation rather than arresting him. 
4 : Deputy Solicitor General Wallace argued the cause for petitioners in both cases. 
5 : Petitioner, a condominium owners' association, filed suit in Maryland state court to compel respondent insurer to pay a $1.1 million judgment it had obtained against respondent's insured, the condominium developer, for numerous defects in the complex. 
6 : Pursuant to Ohio Rev. Code Ann. 

In [ ]:
import re

text_to_deal_with = [
    104, 128, 154, 228, 277, 295
]

# errors on : 251

def contains_date_format(text):
    # Define the regex pattern for [Month Day, Year]
    pattern = r'\[\b(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2},\s+\d{4}\b\]'
    
    # Search for the pattern in the text
    match = re.search(pattern, text)
    return bool(match)

def evaluate_models(document, paragraph_target, file_type):  
    start_time = time.time()
    sentences = fct.sent_segmentation(document, method=method)
    query = select_query(document)
    return query

def select_query(text):
    query = ""
    
    lines = text.split('\n')
    
    for i, line in enumerate(lines):
        line = line.strip()         
        if line.startswith("No.") and i + 1 < len(lines):
            if "Per Curiam" not in lines[i + 1].strip():
                return fct.sent_segmentation(lines[i + 1], "pySBD")[0]
            elif i + 2 < len(lines):
                return fct.sent_segmentation(lines[i + 2], "pySBD")[0]
        if "Syllabus" in line and i + 1 < len(lines):
            query = lines[i + 1]
            
        # if there is a date like [Month day, year], we take :
        # - the next line if it is not a title
        # - else the line following the title
            
        if contains_date_format(line) and i + 2 < len(lines):
            if lines[i + 2].strip() == "I" and i + 3 < len(lines):
                if lines[i + 3].strip() == "A" and i + 4 < len(lines):
                    return fct.sent_segmentation(lines[i + 4], "pySBD")[0]  
                return fct.sent_segmentation(lines[i + 3], "pySBD")[0]  
            return fct.sent_segmentation(lines[i + 2], "pySBD")[0]   
    
    if query != "":
        return fct.sent_segmentation(query, "pySBD")[0]
    else:
        return "law and legal rights"
    
results = []
    
for text_number in tqdm(text_to_deal_with):
    document_json = train_json[text_number]["raw_source"]
    paragraph_target_json = (
        train_json[text_number]['raw_target']['facts_of_the_case'] +
        train_json[text_number]['raw_target']['question'] +
        train_json[text_number]['raw_target']['conclusion']
    )

    clean_path_txt = f'clean_data_txt_save/train_{text_number}.txt'
    clean_document_txt = fct.open_file(clean_path_txt, "txt")

    query = evaluate_models(clean_document_txt, paragraph_target_json, "Cleaned TXT") 
    
    results.append((text_number, query)) 

 33%|███▎      | 2/6 [00:01<00:01,  2.04it/s]

  I


 50%|█████     | 3/6 [00:01<00:01,  2.34it/s]

  I


100%|██████████| 6/6 [00:02<00:00,  3.72it/s]

     The Federal Power Act (FPA or Act), 41 Stat. 1063, as amended, 16 U. S. C. §791a  etseq. , authorizes the Federal Energy Regulatory Commission(FERC or Commission) to superintend the sale of electricity ininterstate commerce and provides that all wholesale-electricityrates must be “just and reasonable,” §824d(a). Under this Court’s Mobile-Sierra  doctrine, FERC must presume that a rate setby “a freely negotiated wholesale-energy contract” meets thestatutory “just and reasonable” requirement.  Morgan StanleyCapital Group Inc.  v.  Public Util. Dist. No. 1 ofSnohomish Cty. , 554 U. S. ___, ___ (2008) (slip op., at 1).“The presumption may be overcome only if FERC concludes that thecontract seriously harms the public interest.”  Ibid.
     As this Court recognized in  Southland Corp.  v.  Keating ,  465 U. S. 1  (1984), theFederal Arbitration Act (FAA or Act), 9 U. S. C. §1  etseq . (2000 ed. and Supp. V), establishes a national policyfavoring arbitration when the parties contract for 

100%|██████████| 6/6 [00:02<00:00,  2.84it/s]


In [38]:
for (i, query) in results:
    print(f'{i} : {query}')

104 : law and legal rights
128 : Respondent Juan Davila is a participant, and respondent Ruby Calad is a beneficiary, in ERISA-regulated employee benefit plans. 
154 : The Beef Promotion and Research Act of 1985 (Beef Act or Act), 99 Stat. 
228 : law and legal rights
277 : The Federal Power Act (FPA or Act), 41 Stat. 
295 : As this Court recognized in  Southland Corp.  


In [ ]:
results.head()

In [ ]:
def highlight_min_max(df):
    styles = pd.DataFrame('', index=df.index, columns=df.columns)

    # Appliquer le style pour les colonnes 'Precision', 'Recall', 'F1-Score'
    for col in ['rouge1', 'rouge2', 'rougeL']:
        # Top 3 maximums et minimums
        top_3_max = df[col].nlargest(3)
        top_3_min = df[col].nsmallest(3)

        # Appliquer le dégradé vert pour les max
        for i in df.index:
            if df[col].iloc[i] in top_3_max.values:
                rank = top_3_max.rank(ascending=False)[top_3_max == df[col].iloc[i]].values[0]
                alpha = 1 - (rank - 1) / 3
                styles.loc[i, col]= f'background-color: rgba(50, 200, 50, {alpha});'

        # Appliquer le dégradé rouge pour les min
        for i in df.index:
            if df[col].iloc[i] in top_3_min.values:
                rank = top_3_min.rank()[top_3_min == df[col].iloc[i]].values[0]
                alpha = 1 - (rank - 1) / 3  
                styles.loc[i, col]= f'background-color: rgba(200, 50, 50, {alpha});'

    # Pour la colonne 'Execution Time', inverser les couleurs (max en rouge, min en vert)
    col = 'Execution Time'
    top_3_max = df[col].nlargest(3)
    top_3_min = df[col].nsmallest(3)

    # Appliquer le dégradé rouge pour les max
    for i in df.index:
        if df[col].iloc[i] in top_3_max.values:
            rank = top_3_max.rank(ascending=False)[top_3_max == df[col].iloc[i]].values[0]
            alpha = 1 - (rank - 1) / 3 
            styles.loc[i, col]= f'background-color: rgba(200, 50, 50, {alpha});'

    # Appliquer le dégradé vert pour les min
    for i in df.index:
        if df[col].iloc[i] in top_3_min.values:
            rank = top_3_min.rank()[top_3_min == df[col].iloc[i]].values[0]
            alpha = 1 - (rank - 1) / 3 
            styles.loc[i, col]= f'background-color: rgba(50, 200, 50, {alpha});'
            
    return styles

styled_df = df.style.apply(highlight_min_max, axis=None)

styled_df

We try pySBD on 20 cleaned documents

In [ ]:
txt_result = []
cleaned_txt_result = []

methods = ['pySBD']

for i in range(0, 20):

    document_json = train_json[i]["raw_source"]
    paragraph_target_json = (
        train_json[i]['raw_target']['facts_of_the_case'] +
        train_json[i]['raw_target']['question'] +
        train_json[i]['raw_target']['conclusion']
    )

    path_txt = 'data_txt_save/train_'+ str(i) + '.txt'
    document_txt = fct.open_file(path_txt, "txt")
    clean_path_txt = 'clean_data_txt_save/train_'+ str(i) + '.txt'
    clean_document_txt = fct.open_file(clean_path_txt, "txt")
    
    r, _ = evaluate_models(document_txt, paragraph_target_json, "TXT")  
    txt_result += r
    r, _ = evaluate_models(clean_document_txt, paragraph_target_json, "Cleaned TXT")  
    cleaned_txt_result += r
    
txt_df = pd.DataFrame(txt_result)  
cleaned_txt_df = pd.DataFrame(cleaned_txt_result)

mean_txt = txt_df[['rouge1', 'rouge2', 'rougeL', 'Execution Time']].mean()
mean_cleaned_txt = cleaned_txt_df[['rouge1', 'rouge2', 'rougeL', 'Execution Time']].mean()


final_df = pd.DataFrame({
    'Text': ['txt', 'cleaned txt'],
    'rouge1': [mean_txt['rouge1'], mean_cleaned_txt['rouge1']],
    'rouge2': [mean_txt['rouge2'], mean_cleaned_txt['rouge2']],
    'rougeL': [mean_txt['rougeL'], mean_cleaned_txt['rougeL']],
    'Execution Time': [mean_txt['Execution Time'], mean_cleaned_txt['Execution Time']]
})

final_df